In [4]:
import sys
!{sys.executable} -m pip install -qU langchain-huggingface

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\Owen\Documents\ia\part2\local-langchain-academy-main\local-langchain-academy-main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Owen\Documents\ia\part2\local-langchain-academy-main\local-langchain-academy-main\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owen\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [10]:
from rdflib import Graph, RDF, OWL, BNode
from typing import Set
g = Graph()
g.parse("./Ontology_Assignment.rdf")

def extract_class_bundle(g: Graph, class_uri) -> Graph:
    """
    Recursively collect all triples that describe or reference a given OWL class.
    Includes nested blank nodes (Restrictions, unionOf, etc.).
    """
    subg = Graph()
    visited: Set = set()

    def recurse(node):
        if node in visited:
            return
        visited.add(node)

        # Triples where node is the subject
        for s, p, o in g.triples((node, None, None)):
            subg.add((s, p, o))
            if isinstance(o, BNode):
                recurse(o)

        # Triples where node is the object
        for s, p, o in g.triples((None, None, node)):
            subg.add((s, p, o))
            if isinstance(s, BNode):
                recurse(s)

    recurse(class_uri)
    return subg


# --- Create bundles for each class ------------------------------------------
print("Extracting class bundles...")

class_bundles = []
for c in g.subjects(RDF.type, OWL.Class):
    subgraph = extract_class_bundle(g, c)
    rdf_text = subgraph.serialize(format="turtle")  # or "xml" if you prefer
    class_bundles.append(rdf_text)
    print(f"Extracted bundle for class: {c}")

print(f"Total class bundles: {len(class_bundles)}")

ontology_blocks = class_bundles

Extracting class bundles...
Extracted bundle for class: N6f6fb4b211644c129408e8853bb33453
Extracted bundle for class: Ndaf342086d014b23af60716bfca10070
Extracted bundle for class: http://www.semanticweb.org/izabo/ontologies/2025/8/Ontology_Assignment#BakingIngredients
Extracted bundle for class: http://www.semanticweb.org/izabo/ontologies/2025/8/Ontology_Assignment#Ingredients
Extracted bundle for class: http://www.semanticweb.org/izabo/ontologies/2025/8/Ontology_Assignment#PizzaBianca
Extracted bundle for class: http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Adoption
Extracted bundle for class: http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Adult
Extracted bundle for class: N4a8376544dcb434eb154f34db4264b43
Extracted bundle for class: http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#AirborneAllergies
Extracted bundle for class: http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Allergies
Extracte

In [16]:
from langchain_core.documents import Document

# Convert ontology blocks (strings) into Documents
docs = [Document(page_content=text) for text in ontology_blocks]

# Add to vector store
document_ids = vector_store.add_documents(docs)

In [18]:
# --- Local LLM for generation (Ollama)
from langchain_ollama import ChatOllama

# Initialize your local LLM (change model name if needed)
llm = ChatOllama(
    model="phi3:mini",
)

In [32]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [36]:
question = "Is Alex allowed to be married at 15?"
k = 3
retrieved_docs = vector_store.similarity_search(question, k=k)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
rendered_prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(rendered_prompt)

In [37]:
answer

AIMessage(content="No, Alex is not allowed to be married at the age of 15 as per this ontology because it defines an 'Adult' class with individuals being considered adults who are over 18 years old. The term marriage here seems associated with the relationship between two Adult entities only; hence, one cannot marry before reaching adulthood according to these definitions.", additional_kwargs={}, response_metadata={'model': 'phi3:mini', 'created_at': '2025-10-16T08:15:56.470665Z', 'done': True, 'done_reason': 'stop', 'total_duration': 38185351600, 'load_duration': 28444200, 'prompt_eval_count': 1778, 'prompt_eval_duration': 3757219800, 'eval_count': 78, 'eval_duration': 6581822300, 'model_name': 'phi3:mini'}, id='run--1a839036-079d-4f0b-8e0f-c4da2b99c68a-0', usage_metadata={'input_tokens': 1778, 'output_tokens': 78, 'total_tokens': 1856})

In [38]:
docs_content

'@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n\n<http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Birth> a owl:Class ;\n    rdfs:subClassOf <http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#LifeEvent> .\n\n[] a owl:AllDisjointClasses ;\n    owl:members ( <http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Birth> <http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Death> <http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Marriage> ) .\n\n\n\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n\n<http://www.semanticweb.org/izabo/ontologies/2025/8/untitled-ontology-8#Death> a owl:Class ;\n    rdfs:subClassOf <http://www.semanticweb.org/izabo/ontologie